**From image-level to pixel-level labeling: A weakly-supervised learning method for identifying aquaculture ponds using iterative anti-adversarial attacks guided by aquaculture features**

Author: Boyi Li, Adu Gong, Jiaming Zhang, Zexin Fu

Update: 2024/07/17


After completing the dataset preparation stage, you should have two folders: one for storing output aquaculture imagery and another for storing output non-aquaculture imagery. These images will be used for pre-training the classification model, generating pseudo-labels, and implementing semantic segmentation.

This is the code used in the paper, and it can be run on Google Colab after editing the code.

#Step 1: Model pre-training
## Input
A folder contains both aquaculture and non-aquaculture imagery. The examplary structure of the folder is as follows:

**/…/WSLM-AQ/inputs/dataset_cls/**

*0: a folder for non-aquaculture imagery*

*1: a folder for aquaculture imagery*

##Output

A folder contains well-trained model and other output files. The sample structure of the folder is as follows:

**/…/WSLM-AQ/models/model_cls/**

*config.yml: config information*

*log_train.csv：model training log (only available when train_mode == 'True')*

*log_val.csv：model validation log (only available when train_mode == 'False')*

*result.jpg: result figure

*model_oa.pth: well-trained model parameters with best oa*

*model_loss.pth: well-trained model parameters with best loss*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
import os
import torch

#todo: Input your working directory
workdir='/content/drive/MyDrive/Colab Notebooks/WSLM-AQ/step'
os.chdir(workdir)

#todo: Set your parameters according to the tips in train_my_classification.py
!python train_my_classification.py \
--name  'model_cls'\
--device 'cuda'\
--dataset 'dataset_cls'\
--work_dir '/content/drive/MyDrive/Colab Notebooks/WSLM-AQ'\
--package_dirs "['/content/drive/MyDrive/Colab Notebooks/WSLM-AQ/packages/packages_dl']"\
--img_ext '.tif'\
--input_channel_list '[0,1,2,3]'\
--num_classes 2\
--transform True\
--train_transform_methods "{'one_hot':{'num_classes':2}}"\
--val_transform_methods "{'one_hot':{'num_classes':2}}"\
--batch_size 8\
--arch 'models.classification.vgg'\
--arch_parameters "{'input_size':(4,256,256),'num_classes':2,'model_name':'vgg19'}"\
--pretrain_backbone False\
--pretrain_path ''\
--train_mode 'True'\
--epochs 20\
--loss 'BinaryCrossEntropyWithLogits'\
--optimizer 'Adam'\
--optimizer_params "{'lr':0.0001}"\
--early_stopping 10\
--best_metric 'loss'

#Step 2: Pseudo-label generation

## Input

img_cam_dir: A folder of input imagery for generating pseudo-labels.

pretrain_dir: Folder for pre-trained model and other files.

##Output

A folder contains output pseudo-labels. The examplary structure of the folder is as follows:

**/…/WSLM-AQ/models/model_cls/cam/**

*result: output imagery with pseudo-labels, which can be used for semantic segmentation*

*config_make_cam.yml：config information*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
import os
import torch

#todo: Input your working directory
workdir='/content/drive/MyDrive/Colab Notebooks/WSLM-AQ/step'
os.chdir(workdir)

#todo: Set your parameters according to the tips in make_cam.py
!python make_cam.py \
--device 'cuda'\
--package_dirs "['/content/drive/MyDrive/Colab Notebooks/WSLM-AQ/packages/packages_dl']"\
--pretrain_dir '/content/drive/MyDrive/Colab Notebooks/WSLM-AQ/models/model_cls'\
--model_name 'model_loss.pth'\
--outputdir_name 'cam'\
--img_cam_dir '/content/drive/MyDrive/Colab Notebooks/WSLM-AQ/inputs/dataset_cls/train'\
--output_type '.tif'\
--classes_pos "[1]"\
--water_channels -2\
--label_channels -1\
--scales '(1.0, 0.5, 0.75, 1.25, 1.5, 1.75, 2.0)'\
--cam 'models.segmentation.gradcam'\
--target_layer "['layer5.12']"\
--target_layer_weight "[1]"\
--adv_iter 20\
--score_th 0.5\
--AD_coeff 7\
--AD_stepsize 0.08\
--flip_augmentation 'True'\
--use_water 'True'\
--water_threshold 0.3\
--suppressing_classes 'True'\
--add_discriminative 'True'\
--water_for_discriminative 'True'\
--non_water_for_discriminative 'True'\
--AD_coeff2 7

#Step 3: Semantic segmentation

## Input

A folder contains both aquaculture and non-aquaculture imagery. The sample structure of the folder is as follows:

**/…/WSLM-AQ/inputs/dataset_seg/**

*0: a folder for non-aquaculture imagery with labels*

*1: a folder for aquaculture imagery with labels*

A folder contains well-trained model and other output files. The examplary structure of the folder is as follows:

**/…/WSLM-AQ/models/model_seg/**

*config.yml: config information*

*log_train.csv：model training log (only available when train_mode == 'True')*

*log_val.csv：model validation log (only available when train_mode == 'False')*

*result.jpg: result figure*

*model_iou.pth: well-trained model parameters with best oa (only available when best_metric == 'iou')*

*model_loss.pth: well-trained model parameters with best oa (only available when best_metric == 'loss')*

*output: output labels (only available when train_mode == 'False')*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
import os
import torch

#todo: Input your working directory
workdir='/content/drive/MyDrive/Colab Notebooks/WSLM-AQ/step'
os.chdir(workdir)

#todo: Set your parameters according to the tips in train_my_segmentation.py
!python train_my_segmentation.py \
--name  'model_seg'\
--device 'cuda'\
--dataset 'dataset_cls'\
--work_dir '/content/drive/MyDrive/Colab Notebooks/WSLM-AQ'\
--package_dirs "['/content/drive/MyDrive/Colab Notebooks/WSLM-AQ/packages/packages_dl']"\
--img_ext '.tif'\
--input_channel_list '[0,1,2,3]'\
--pos_classes '[1]'\
--neg_classes '[0]'\
--transform 'True'\
--train_transform_methods "{'one_hot_segmentation_binary':{'test':None}}"\
--val_transform_methods "{'one_hot_segmentation_binary':{'test':None}}"\
--batch_size_train 8\
--batch_size_val 1\
--arch 'models.segmentation.unet_vgg'\
--arch_parameters "{'input_size':(4,256,256),'num_classes':2,'model_name':'vgg19'}"\
--pretrain_backbone 'True'\
--pretrain_path '/content/drive/MyDrive/Colab Notebooks/WSLM-AQ/models/model_cls/model_loss.pth'\
--train_mode 'True'\
--epochs 40\
--optimizer 'Adam'\
--optimizer_params "{'lr':0.001}"\
--loss 'BinaryCrossEntropyWithLogits'\
--early_stopping 10\
--neg_for_train 'True'\
--update 'True'\
--update_epoch 20\
--best_metric 'loss'